In [1]:
using Pkg
Pkg.activate("./VisualisationToolKit")
Pkg.instantiate()

  Activating project at `~/Documents/Robin/ut4m-cardio/Julia/Script_Robin/VisualisationToolKit`
   Installed ConstructionBase ─ v1.5.8
   Installed InverseFunctions ─ v0.1.17
   Installed Accessors ──────── v0.1.38
   Installed MultipleTesting ── v0.6.0
   Installed Roots ──────────── v2.2.1
   Installed HypothesisTests ── v0.11.3
Precompiling project...
    739.4 ms  ✓ CommonSolve
    743.5 ms  ✓ CompositionsBase
    746.8 ms  ✓ InverseFunctions
    743.3 ms  ✓ ConstructionBase
    977.2 ms  ✓ Combinatorics
    862.0 ms  ✓ InverseFunctions → InverseFunctionsDatesExt
    874.3 ms  ✓ LogExpFunctions → LogExpFunctionsInverseFunctionsExt
    916.3 ms  ✓ Unitful → InverseFunctionsUnitfulExt
    918.3 ms  ✓ InverseFunctions → InverseFunctionsTestExt
    919.8 ms  ✓ CompositionsBase → CompositionsBaseInverseFunctionsExt
    566.4 ms  ✓ ConstructionBase → ConstructionBaseLinearAlgebraExt
    574.8 ms  ✓ Unitful → ConstructionBaseUnitfulExt
    574.8 ms  ✓ StatsFuns → StatsFunsInverseFunctions

In [2]:
Pkg.add("HypothesisTests")
Pkg.add("MultipleTesting")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/Documents/Robin/ut4m-cardio/Julia/Script_Robin/VisualisationToolKit/Project.toml`
  No Changes to `~/Documents/Robin/ut4m-cardio/Julia/Script_Robin/VisualisationToolKit/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/Robin/ut4m-cardio/Julia/Script_Robin/VisualisationToolKit/Project.toml`
  No Changes to `~/Documents/Robin/ut4m-cardio/Julia/Script_Robin/VisualisationToolKit/Manifest.toml`


In [3]:
using XLSX,DataFrames
include("../preprocessing.jl")
data = XLSX.readxlsx("../../Ressources/Données complètes UT4M final120419 avec Data cardio.xlsx")["Tableau_gl"]
data = data["A1:SD80"]

df = DataFrame(data[2:end, 2:end], Symbol.(data[1, 2:end]), makeunique=true)
CODE_SUJET = DataFrame([:"CODE_SUJET" => data[2:end,1]])
df = hcat(CODE_SUJET, df)
df = replaceDF(df,-1,missing)
mesures = df[:,24:213]
mesures_cardiaques = df[:,267:end]
res = colToRow(mesures_cardiaques)
res = addCOURSE_SUJET(df, res)

Row,CODE_SUJET,COURSE,TIME,STDi,DTDVGI,At,VOLODI,TDM,PP,VOL_OGI,ITVPUL,Et,E/A_1,Ea,STSi,FR,VOL_OG,DTDVG,STS,VES,VOLOD,TD,E/Ea,VTDVGI,S,PAPS,PAPMAX,FRVD,ITV Ao,STD,MVG,Qc,VTDVG,PAD,VOLOGI,H/R,MVGI,E/A,FC,CC VG,ITV_PULM,PAM,VTSVGI,FE,VTSVG,PAS,IC,VOL_OD,St,FE2D,VOLOG,A,E,VOL_ODI,DTS
,Int64,Any,String,Any,Any,Any,Any,Any,Any,Any,Any,Any,Missing,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1,100,-2,12.3529,25.6684,28,missing,203,9,27.2727,missing,66,missing,18,6.57754,0.3125,51,48,12.3,76,missing,246,3.72222,72.1925,10,missing,missing,0.467532,13.8,23.1,158.223,3.61909,135,80,missing,0.395833,84.6112,2.16129,58,2.4,21,97.3333,31.5508,57,59,132,1.93534,46,16,58,missing,31,67,24.5989,33
2,2,100,-2,11.5642,25.1397,25,missing,255,10,24.581,missing,52,missing,16,5.36313,0.444444,44,45,9.6,69,missing,178,3.5625,61.4525,10,missing,28,0.536232,21.6,20.7,152.673,6.87735,110,68,missing,0.444444,85.292,1.5,60,2.6,18.9,84.3333,22.905,75,41,117,3.8421,59,14,63,missing,38,57,32.9609,25
3,3,100,-2,12,26.1111,26,missing,215,10,33.3333,missing,53,missing,17,5.27778,0.297872,60,47,9.5,47,missing,199,3.88235,50.5556,10,missing,missing,0.560185,17.1,21.6,163.855,3.57334,91,74,missing,0.425532,91.0304,1.29412,55,2.2,17,91.3333,24.4444,56,44,126,1.98519,42,16,52,missing,51,66,23.3333,33
4,4,100,-2,11.5686,28.7582,17,missing,201,9,28.1046,missing,48,missing,missing,5.62092,0.363636,43,44,8.6,43,missing,237,missing,52.9412,9,missing,missing,0.514124,20.5,17.7,127.416,3.13707,81,65,missing,0.409091,83.2787,1.78788,54,1.9,19.2,77.6667,24.8366,66,38,103,2.05037,32,10,53,missing,33,59,20.915,28
5,5,100,-2,13.6316,30.5263,26,missing,185,12,54.2105,missing,50,missing,18,6.84211,0.396552,103,58,13,65,missing,284,4.22222,64.7368,10,missing,missing,0.498069,25.5,25.9,263.651,6.25547,123,68,missing,0.37931,138.764,1.65217,50,2.5,19.6,86.3333,30.5263,71,58,123,3.29235,75,13,53,missing,46,76,39.4737,35
6,6,100,-2,12.1026,29.2308,20,missing,327,10,36.9231,missing,38,missing,9,7.64103,0.368421,72,57,14.9,73,missing,203,4.55556,68.7179,11,missing,17,0.368644,15.8,23.6,240.747,4.95922,134,62,missing,0.368421,123.46,1.36667,51,2.8,missing,78,31.2821,65,61,110,2.54319,77,12,54,missing,30,41,39.4872,36
7,7,100,-2,13.6585,29.2683,missing,missing,214,11,41.9512,missing,missing,missing,21,8.4878,0.383333,86,60,17.4,75,missing,200,4.04762,70.2439,10,missing,25,0.378571,31.9,28,262.448,7.21195,144,77,missing,0.35,128.023,1.77083,50,2.4,22.2,97,33.6585,68,69,137,3.51803,76,15,52,missing,48,85,37.0732,37
8,8,100,-2,12.1341,34.7561,31,missing,206,9,50,missing,54,missing,16,6.40244,0.315789,82,57,10.5,69,missing,174,4.5,74.3902,9,missing,28,0.472362,24.8,19.9,196.92,5.66636,122,68,missing,0.315789,120.073,1.6,66,2.1,19.1,82.6667,32.3171,59,53,112,3.4551,60,14,57,missing,45,72,36.5854,39
9,9,100,-2,9.53846,25.641,24,missing,224,9,32.3077,missing,55,missing,17,5.94872,0.32,63,50,11.6,75,missing,228,4.64706,69.7436,10,missing,missing,0.376344,25.9,18.6,169.32,5.02107,136,70,missing,0.38,86.8308,1.68085,56,2.1,19.1,87,31.2821,60,61,121,2.57491,45,14,55,missing,47,79,23.0769,34


In [4]:
using DataFrames, CategoricalArrays, GLM

function fit_linear_model(df::DataFrame, val_study::String)
    # Filtrer les lignes où la colonne TIME est égale à "-2" et "1"
    filtered_rows_PRE = filter(row -> row.TIME == "-2", df)
    filtered_rows_POST = filter(row -> row.TIME == "1", df)

    # Sélectionner uniquement les colonnes TIME et val_study
    filtered_df_PRE = filtered_rows_PRE[:, [Symbol(val_study), :COURSE]]
    rename!(filtered_df_PRE, Symbol(val_study) => "PRE")

    filtered_df_POST = filtered_rows_POST[:, [Symbol(val_study)]]
    rename!(filtered_df_POST, Symbol(val_study) => "POST")
    

    # Combiner les DataFrames
    combined_df = hcat(filtered_df_PRE, filtered_df_POST)
    combined_df = dropmissing(combined_df)
    
    
    # Convertir les colonnes en Float64 et catégoriser COURSE
    
    combined_df[!, Symbol("PRE")] = convert.(Float64, combined_df[!, Symbol("PRE")])
    combined_df[!, Symbol("POST")] = convert.(Float64, combined_df[!, Symbol("POST")])
    combined_df.COURSE = string.(combined_df.COURSE)
    combined_df.COURSE = categorical(combined_df.COURSE)
    # Ajuster le modèle linéaire
    modelsimple = lm(@formula(PRE ~ POST + COURSE), combined_df)
    modelinteraction = lm(@formula(PRE ~ POST * COURSE), combined_df)
    return modelsimple,modelinteraction
    
    
end

fit_linear_model (generic function with 1 method)

In [5]:
fit_linear_model(res, "FE2D")

(StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

PRE ~ 1 + POST + COURSE

Coefficients:
──────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)   41.9148      7.55523    5.55    <1e-05  26.7397    57.0899
POST           0.240602    0.138384   1.74    0.0883  -0.037351   0.518555
COURSE: 160    1.24443     2.28227    0.55    0.5880  -3.33965    5.82851
COURSE: 40    -0.16792     2.29392   -0.07    0.9419  -4.77539    4.43955
COURSE: 4_40   0.823128    2.35452    0.35    0.7281  -3.90606    5.55232
──────────────────────────────────────────────────────────────────────────, StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Floa

In [6]:
using HypothesisTests
function mean_test(df::DataFrame,val_study::String)
    # Filtrer les lignes où la colonne TIME est égale à "-2" et "1"
    filtered_rows_PRE = filter(row -> row.TIME == "-2", df)
    filtered_rows_POST = filter(row -> row.TIME == "1", df)

    # Sélectionner uniquement les colonnes TIME et val_study
    filtered_df_PRE = filtered_rows_PRE[:, [Symbol(val_study), :COURSE]]
    rename!(filtered_df_PRE, Symbol(val_study) => "PRE")

    filtered_df_POST = filtered_rows_POST[:, [Symbol(val_study)]]
    rename!(filtered_df_POST, Symbol(val_study) => "POST")
    

    # Combiner les DataFrames
    combined_df = hcat(filtered_df_PRE, filtered_df_POST)
    combined_df = dropmissing(combined_df)
    
    
    # Convertir les colonnes en Float64 et catégoriser COURSE
    
    combined_df[!, Symbol("PRE")] = convert.(Float64, combined_df[!, Symbol("PRE")])
    combined_df[!, Symbol("POST")] = convert.(Float64, combined_df[!, Symbol("POST")])
    combined_df.COURSE = string.(combined_df.COURSE)
    combined_df.COURSE = categorical(combined_df.COURSE)

    course_name = unique(filter(x -> x != "4_40", combined_df.COURSE))
    pvalues = []
    for course in course_name
        course_df = filter(row -> row.COURSE == course, combined_df)
        x = course_df[!,:PRE]
        y = course_df[!,:POST]
        push!(pvalues,pvalue(OneSampleTTest(x, y)))
    end
    return pvalues
    
end

mean_test (generic function with 1 method)

In [30]:
using DataFrames
col_names = ["100", "40", "160"]
variable_interest = ["VTDVGI", "VTSVGI", "FE2D", "STDi", "STSi", "VES"]
results = DataFrame(COURSE=String[], PValue=Float64[], Column=String[])

df_results = DataFrame(COURSE=variable_interest)
for (i, col_name) in enumerate(col_names)
    df_results[!, Symbol(col_name)] = [mean_test(res, var)[i] for var in variable_interest]
end

# Inverser les lignes et les colonnes
df_results_transposed = permutedims(df_results, :COURSE)
df_results_transposed

Row,COURSE,VTDVGI,VTSVGI,FE2D,STDi,STSi,VES
,String,Float64,Float64,Float64,Float64,Float64,Float64
1,100,0.203544,0.0743086,0.370328,0.944651,0.526718,0.979035
2,40,0.0682626,0.850616,0.497404,0.406247,0.0424831,0.101765
3,160,0.612307,0.954117,0.71111,0.000695863,8.1812e-5,0.556929


In [29]:
function create_corr_pval_dataframe(pvalues::Vector{Float64}, method)
    pval_adj = adjust(pvalues, method)
    matcorrected = fill(0.0, 3, 6)
    compteur = 1
    for i in 1:3
        for j in 1:6
            matcorrected[i,j] = pval_adj[compteur]
            compteur += 1
        end
    end
    matcorrected = hcat(["100", "40", "160"], matcorrected)
    col_name = ["COURSE", "VTDVGI", "VTSVGI", "FE2D", "STDi", "STSi", "VES"]
    df_final = DataFrame(matcorrected, Symbol.(col_name))
    return df_final
end

create_corr_pval_dataframe (generic function with 1 method)

In [36]:
df_pval = df_results_transposed[!, Not(:COURSE)]
vector_pvalues = []
for i in 1:size(df_pval, 1)
    for j in 1:size(df_pval, 2)
        push!(vector_pvalues, df_pval[i, j])
    end
end
vector_pvalues = convert.(Float64,vector_pvalues)
display(create_corr_pval_dataframe(vector_pvalues, Hochberg()))
display(create_corr_pval_dataframe(vector_pvalues, BenjaminiYekutieli()))
display(create_corr_pval_dataframe(vector_pvalues, BenjaminiHochberg()))

Row,COURSE,VTDVGI,VTSVGI,FE2D,STDi,STSi,VES
,Any,Any,Any,Any,Any,Any,Any
1,100,0.979035,0.979035,0.979035,0.979035,0.979035,0.979035
2,40,0.979035,0.979035,0.979035,0.979035,0.67973,0.979035
3,160,0.979035,0.979035,0.979035,0.0118297,0.00147262,0.979035


Row,COURSE,VTDVGI,VTSVGI,FE2D,STDi,STSi,VES
,Any,Any,Any,Any,Any,Any,Any
1,100,1.0,0.934979,1.0,1.0,1.0,1.0
2,40,0.934979,1.0,1.0,1.0,0.890898,1.0
3,160,1.0,1.0,1.0,0.021889,0.00514695,1.0


Row,COURSE,VTDVGI,VTSVGI,FE2D,STDi,STSi,VES
,Any,Any,Any,Any,Any,Any,Any
1,100,0.5234,0.267511,0.812495,0.979035,0.835394,0.979035
2,40,0.267511,0.979035,0.835394,0.812495,0.254899,0.305296
3,160,0.84781,0.979035,0.914284,0.00626277,0.00147262,0.835394
